In [31]:
import numpy as np
import pandas as pd
# not compatible with pandas 1.1.0
# pip3 install --user -U pandas==1.0.5
# see https://github.com/limix/pandas-plink/issues/18
from pandas_plink import read_plink

from IPython.display import display, HTML

In [2]:
prefix = '/home/arash/temp/BitEpi/sampleData/bfile'
prefix

'/home/arash/temp/BitEpi/sampleData/bfile'

In [3]:
# Return a genotype datafream generated from plink bfile (BDF: Bfile DataFrame)
def BDF(prefix):
    (bim, fam, bed) = read_plink(prefix, verbose=True)
    bdf = pd.DataFrame(bed.compute().astype('int8')).join(bim[['snp']])\
    .set_index('snp').append(fam.trait.astype('int8')).transpose().astype('category')
    bdf['cnt']=1
    return bdf

In [44]:
# Return the Contingency Table (CT) with sum (s) and row weight (w) given a number of SNPs and Bfile Data Frame
def CT(bdf, SNPs, pheno = 'trait'):
    ct = bdf.groupby([pheno]+SNPs).count()[['cnt']]
    ctrl = ct.loc[ct.index.get_level_values(pheno) == 1].droplevel(level=0).rename(columns={"cnt": "ctrl"})
    case = ct.loc[ct.index.get_level_values(pheno) == 2].droplevel(level=0).rename(columns={"cnt": "case"})
    ctx = ctrl.join(case)
    ctx['s'] = ctx.sum(axis=1)
    total = ctx['s'].sum()
    ctx['w'] = ctx['s']/total
    return ctx.fillna(0)

In [50]:
# Return Weighted Average Purity (WAP) given a contingency table
def WAP(ct):
#     ct['x']=(ct.ctrl/ct.s)**2
#     ct['y']=(ct.case/ct.s)**2
#     ct['z']=ct.x + ct.y
#     ct['zz']=ct.z * ct.w
#     print("=========================")
#     print(ct)
#     display(HTML(ct.to_html()))
#     print(ct.zz.sum())
#     print("=========================")
    # *** This does not handle division by 0
    #print(ct)
    return ((((ct.ctrl/ct.s)**2) + ((ct.case/ct.s)**2)) * ct.w).sum()

In [51]:
# Return Mximum Lower Order WAP (MLOWAP). Lower Order means to exclude 1 SNP from combination
def MLOWAP(ct, SNPs):
    lowaps = list()
    for i in range(0, len(SNPs)):
        lo = SNPs.copy()
        del lo[i]
        #print(lo)
        ctx = ct.groupby(lo).sum()
        lowaps.append(WAP(ctx))
    return max(lowaps)

In [52]:
#return Alpha and Beta (BitEpi) given contingency table
def AB(ct):
    #display(HTML(ct.to_html()))
    wap = WAP(ct)
    mlowap = MLOWAP(ct,SNPs)
    return {'beta':wap, 'alpha':(wap - mlowap)}

In [73]:
def RP(ct):
    # pc: probability of being case
    # pc = np.random.uniform(0, 1, ct.shape[0])
    pc = np.random.normal(0.5, 0.25, ct.shape[0])
    pc[pc > 1] = 1
    pc[pc < 0] = 0
    ct['pc'] = pc
    ct.case = (ct.s * ct.pc).astype('int32')
    ct.ctrl = (ct.s - ct.case).astype('int32')
    ct.drop(['pc'], axis=1, inplace=True)

In [100]:
bdf = BDF(prefix)
SNPs=['N21','N56']#['M0P1','M0P2']
ct = CT(bdf, SNPs)
ab = AB(ct)  
print(ab)
# rab for AB with random phenotype
rab = list()
for i in range(100):
    bdf['RandomPheno'] = np.random.choice([1,2], bdf.shape[0])
    ct = CT(bdf, SNPs)
    rab.append(AB(ct))
    
ac = len([r for r in rab if r['alpha'] > ab['alpha']])
bc = len([r for r in rab if r['beta'] > ab['beta']])

print(ac, bc)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 78.36it/s]


{'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517}
0 0


In [101]:
rab

[{'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.0012293219338721517},
 {'beta': 0.5029188128210933, 'alpha': 0.00122932193387

In [98]:
bdf['RandomPheno'] = np.random.choice([1,2], bdf.shape[0])
bdf

snp,N0,N1,N2,N3,N4,N5,N6,N7,N8,N9,...,N93,N94,N95,N96,N97,M0P1,M0P2,trait,cnt,RandomPheno
0,2,2,2,1,2,2,1,2,2,0,...,2,1,2,1,2,2,2,2,1,1
1,2,1,2,1,2,2,1,0,1,2,...,0,1,2,1,2,1,2,2,1,1
2,2,1,2,2,2,1,2,2,1,2,...,1,0,1,2,1,2,2,2,1,1
3,2,0,1,2,2,2,1,2,2,1,...,2,1,1,2,1,2,2,2,1,1
4,2,0,2,0,2,2,2,2,0,2,...,1,2,2,0,2,2,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2,1,2,2,2,2,2,1,2,0,...,2,2,1,1,2,2,2,1,1,1
1996,2,2,2,1,2,2,2,1,1,1,...,1,0,2,1,2,1,2,1,1,2
1997,2,2,2,1,2,1,2,2,0,0,...,1,2,2,1,2,2,2,1,1,2
1998,2,1,2,0,2,1,2,2,0,2,...,1,1,2,0,2,2,1,1,1,2


In [94]:
bdf = BDF(prefix)
SNPs=['M0P1','M0P2']
ct = CT(bdf, SNPs)
ab = AB(ct)  
print(ab)
# rab for AB with random phenotype
rab = list()
for i in range(1000):
    RP(ct)
    rab.append(AB(ct))
    
ac = len([r for r in rab if r['alpha'] > ab['alpha']])
bc = len([r for r in rab if r['beta'] > ab['beta']])

print(ac, bc)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 165.71it/s]


{'beta': 0.504218620099264, 'alpha': 0.0035141920889357747}
741 992
